In [1]:
from langchain_community.document_loaders import PDFPlumberLoader, PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
import faiss
from langchain_community.vectorstores import FAISS
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_ollama import OllamaEmbeddings, ChatOllama
import dspy
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_core.documents import Document


/root/miniconda3/envs/patient_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_document_from_pdf(file_path):
    document_loader = PyPDFLoader(file_path)
    docs = document_loader.load()
    return docs[0].page_content

In [4]:
docs = get_document_from_pdf(r'/workspace/LLM/Ankit-llm/prompt_optim/COM virtual patient data - Copy.pdf')

In [5]:
docs

"Scenario 1 : \nA 24-year-old male, Mr Saravanan, who is a driver by occupation residing in Karaikal, has come with \ncomplaints of right side ear discharge for 7 years. He was apparently normal before 7 years and had \nepisodes of ear discharge for 7 years with insidious onset, each episode lasts for 3-7 days, with \nmoderate to profuse ear discharge, discharge is mucopurulent, cream in color, not fouling and not \nblood-stained. Each episode is either aggravated by URI(upper respiratory infection) or head bath and \nrelieved completely by medications. \nHe has some difficulty in hearing in his right ear. He has no history of vertigo, tinnitus, headache, \nfever, vomiting, malaise, lethargy, facial asymmetry, ear pain, retroorbital pain, diplopia, or vision \nproblems. \nHe has no history of trauma to the ear. He has no nasal obstruction, nasal discharge, sneezing, itching \nover the nose, epistaxis, smell disturbances, throat pain, cough, swallowing difficulty, or breathing \ndifficu

In [6]:
dspy.configure(lm=dspy.LM('ollama_chat/gemma3:12b', api_base='http://localhost:11434', api_key='',temperature=0.6))

In [13]:
#================Input Conversion Signature=================
class InputConverter(dspy.Signature):
    """
    Role: You are an expert translator, convert the input language into English language.
    
    Instructions:
        1. Translate the input Hinglish text into English.
        2. Ensure the translation is contextually accurate and maintains the original meaning.
        3. Only provide the translated text without any additional commentary or explanation.
        4. Use proper English grammar and vocabulary suitable for a patient-doctor conversation.
        5. Do NOT transliterate; write in proper English.
    """
    question: str = dspy.InputField(description="Doctor's question in Hinglish")
    converted_question: str = dspy.OutputField(description="Doctor's question in English")
    
class OutputConverter(dspy.Signature):5
    """
    Role: You are an expert translator, convert English language into Hindi language with Devanagari script.
    
    Instructions:
        1. Translate the input English text into Hindi using Devanagari script.
        2. Ensure the translation is contextually accurate and maintains the original meaning.
        3. Only provide the translated text without any additional commentary or explanation.
        4. Use proper Hindi grammar and vocabulary suitable for a patient-doctor conversation.
        5. Use the context of medical conversation while translating for appropriate terminology and meaning.
    Critical:
        -Do NOT transliterate; write in proper Hindi (Devanagari script).
        - STRICTLY avoid using any other script or language.
    """
    response_input: str = dspy.InputField(description="Patient's answer in English")
    context: str = dspy.InputField(description="Patient’s medical history and current condition")
    response: str = dspy.OutputField(description="Patient's answer in Hindi (Devanagari script)")

# ========== QAPatient Signature ==========
class QAPatient(dspy.Signature):
    """
    Role: Simulate a realistic patient in a doctor-patient conversation.

    Instructions:
        1. Respond only to the doctor's question using the given context and retrieved history.
        2. Plan your answer before responding, ensuring relevance to the question.
        3. Answer should directly address the doctor's question.
        4. Keep answers brief and to the point.
        5. Avoid medical jargon; use layman's terms.
        6. Do NOT add extra details to the response unless explicitly asked.
        7. Maintain consistency with patient profile and symptoms.
    Critical:
        - STRICTLY follow context and retrieved history.
        - NEVER invent or assume any medical details.
    """
    question: str = dspy.InputField(description="Doctor's question in English")
    context: str = dspy.InputField(description="Patient’s medical history and current condition")
    retrieved_history: Optional[str] = dspy.InputField(description="Relevant conversation history snippets")
    response: str = dspy.OutputField(description="Patient’s response in English")


# ========== HistoryRetriever Signature ==========
class HistoryRetriever(dspy.Signature):
    """
    Role: Retrieve the most relevant parts of the conversation history to help answer the doctor’s question.

    Instructions:
        1. Understand the doctor’s question within the current medical context.
        2. Retrieve only relevant past exchanges helpful to form the patient’s next response.
        3. Rank all retrieved history items on a scale of 1–10 for relevance.
        4. Return the top 3 ranked history items as a single string summary.
        5. Be concise and remove unnecessary details.
    """
    question: str = dspy.InputField(description="Doctor's question in Hinglish")
    context: str = dspy.InputField(description="Patient’s medical history and condition")
    history: Optional[dspy.History] = dspy.InputField(description="Complete conversation history object")
    retrieved_history: str = dspy.OutputField(description="Top 3 most relevant conversation history entries (ranked and summarized)")


# ========== ResponseJudge Signature ==========
class ResponseJudge(dspy.Signature):
    """
    Role: Evaluate the quality and accuracy of the patient’s response.

    Evaluation Criteria:
        1. Does the patient’s response directly answer the doctor’s question?
        2. Is it consistent with the provided context and medical history?
        3. Does it align with retrieved conversation history?
        4. Is it written entirely in Hindi (Devanagari script)?
        5. Is the response tone appropriate for a patient (not robotic or overly formal)?

    Output:
        - Assign a score between 1 to 5 (1 = poor, 5 = excellent)
    """
    question: str = dspy.InputField(description="Doctor's question in Hinglish")
    response: str = dspy.InputField(description="Patient’s generated response in Hindi (Devanagari script)")
    context: str = dspy.InputField(description="Patient’s medical history and context")
    retrieved_history: Optional[str] = dspy.InputField(description="Retrieved conversation history used for response generation")
    response_rating: int = dspy.OutputField(description="Quality rating between 1 (poor) and 5 (excellent)", ge=1, le=5)


# ========== FailCase Signature ==========
class FailCase(dspy.Signature):
    """
    Role: Provide a fallback patient response when confidence or rating is too low.

    Output:
        - A short Hindi sentence politely saying the patient is unable to answer.
    Example Output:
        "मुझे इस सवाल का जवाब ठीक से नहीं पता डॉक्टर साहब।"
    """
    response: str = dspy.OutputField(description="Fallback response in Hindi (Devanagari script)")

In [14]:
history = dspy.History(messages=[])

In [ ]:
# class RAGmodule(dspy.Module):
#     def __init__(self):
#         self.response = dspy.ChainOfThought(QAPatient)
#         self.response_rating = dspy.ChainOfThought(ResponseJudge)
#         self.retrieve_history = dspy.ChainOfThought(HistoryRetriever)
#         self.response_failure = dspy.Predict(FailCase)
#         self.output_converter = dspy.ChainOfThought(OutputConverter)
#         self.input_converter = dspy.Predict(InputConverter)
#     def forward(self,query:str):
#         context = docs
#         print(f"Context:\n {context}")
#         input_question = self.input_converter(question=query)
#         print(f"Converted Question:\n {input_question.converted_question}")
#         retrieved_history = self.retrieve_history(question=input_question,context=context,history=history)
#         print(f"history:\n {retrieved_history}")
#         retrieved_history = retrieved_history.retrieved_history
#         response = self.response(question=input_question,context=context,retrieved_history=retrieved_history)
#         print(f"Raw Response:\n {response.response}")
#         converted_response = self.output_converter(response=response.response,context=context)
#         responce_rating = self.response_rating(question=input_question,response=converted_response.converted_response,context=context,retrieved_history=retrieved_history)
#         final_respnse = converted_response.converted_response
#         print(f"Response rating:\n{responce_rating}")
#         if responce_rating.response_rating<4:
#             final_respnse = self.response_failure().fail_response
#         history.messages.append({"question": input_question, "Patient":response.response})
#         return final_respnse

In [15]:
class RAGmodule(dspy.Module):
    def __init__(self):
        self.response = dspy.ChainOfThought(QAPatient)
        self.response_rating = dspy.ChainOfThought(ResponseJudge)
        self.retrieve_history = dspy.ChainOfThought(HistoryRetriever)
        self.response_failure = dspy.Predict(FailCase)
        self.output_converter = dspy.ChainOfThought(OutputConverter)
        self.input_converter = dspy.Predict(InputConverter)
        self.prev_sid = 1
    def forward(self,sid:int=0,context:str=None,question:str=None):
        if context==None and sid==0:
            context = docs
        else:
            print(f"SID: {sid} PrevSID {self.prev_sid}")
            if sid != self.prev_sid:
                history.messages.clear()
                self.prev_sid = sid
            print(f"History: {history.messages}")
        input_question = self.input_converter(question=question)
        retrieved_history = self.retrieve_history(question=input_question.converted_question,context=context,history=history)
        retrieved_history = retrieved_history.retrieved_history
        response = self.response(question=input_question.converted_question,context=context,retrieved_history=retrieved_history)
        converted_response = self.output_converter(response_input=response.response,context=context)
        responce_rating = self.response_rating(question=question,response=converted_response.response,context=context,retrieved_history=retrieved_history)
        final_response = converted_response
        if responce_rating.response_rating<4:
            fail_response = self.response_failure()
            return fail_response
        history.messages.append({"question": input_question.converted_question, "Patient": response.response})
        return final_response

In [16]:
rag = RAGmodule()
rag.load("Optimized_patient_rag12b-2.json")

In [18]:
response = rag(question='Aap aaj yahan kis kaam se aaye hain?')
print(response.response)

मेरे दाएं कान से लगभग 7 सालों से डिस्चार्ज हो रहा है। यह आता और जाता रहता है।


In [38]:
print(history.messages)

[{'question': Prediction(
    reasoning='The input question is in Hinglish, a mix of Hindi and English. "Aap" is a Hindi word meaning "you" (formal). The question asks about the reason for the patient\'s visit. I will translate the Hindi portion into English while maintaining the sentence structure.',
    converted_question='What brings you here today?'
), 'Patient': "I've had ear discharge for 7 years, it’s cream-colored, and gets better with medication."}]


In [15]:
history.messages.clear()
print(history.messages)

[]


In [19]:
response = rag(question='Kya aap mujhe apne symptoms ke baare mein thoda aur bata sakte hain?')
print(response.response)

मेरे दाएं कान से लगभग 7 सालों से स्राव हो रहा है। यह एक क्रीमी, पस जैसा स्राव है जो 3 से 7 दिनों तक रहता है। कभी-कभी मुझे उस कान में सुनने में परेशानी होती है। यह ठंडा लगने या बाल धोने के बाद बदतर हो जाता है, लेकिन दवा से यह ठीक हो जाता है।


In [20]:
response = rag(question='Yeh problem lagaatar rehti hai ya aati-jaati rehti hai?')
print(response.response)

यह आता-जाता रहता है।


In [39]:
print(history.messages)

[{'question': 'Aap aaj yahan kis kaam se aaye hain?', 'Patient': 'मुझे दाएं कान से पस होने की शिकायत है, डॉक्टर साहब।'}, {'question': 'Kya aap mujhe apne symptoms ke baare mein thoda aur bata sakte hain?', 'Patient': 'हाँ, पिछले सात सालों से मुझे दाएं कान से पस बह रहा है, जो कभी-कभी बढ़ जाता है।'}]


In [21]:
response = rag(question='Yeh behtar ho raha hai, kharaab ho raha hai, ya waisa hi hai?h')
print(response.response)

यह आता-जाता रहता है।


In [22]:
response = rag(question='Kya aapko koi pehle se medical condition hai jaise diabetes ya high blood pressure?')
print(response.response)

नहीं, मेरे पास इनमें से कोई भी नहीं है।


In [23]:
response = rag(question='Appki umar kitni hai')
print(response.response)

मैं 24 साल का हूँ।


In [24]:
response = rag(question="Aap ka naam kya h")
print(response.response)

मेरा नाम सरवणन है।


In [25]:
response = rag(question='Kay aap ke ghar ke paas construction ka kaam chal raha h?')
print(response.response)

नहीं।


In [27]:
response = rag(question='Kay aap ko swimming aati h?')
print(response.response)

मुझे इस सवाल का जवाब ठीक से नहीं पता डॉक्टर साहब।


In [29]:
response = rag(question='Kay ko d-ram aur s-ram ke beech ka farak pata h?')
print(response.response)

मुझे इस सवाल का जवाब ठीक से नहीं पता डॉक्टर साहब।


In [41]:
response = rag(query='Kay ko stack aur queue ke beech ka farak pata h?')
print(response)

Context:
 Scenario 1 : 
A 24-year-old male, Mr Saravanan, who is a driver by occupation residing in Karaikal, has come with 
complaints of right side ear discharge for 7 years. He was apparently normal before 7 years and had 
episodes of ear discharge for 7 years with insidious onset, each episode lasts for 3-7 days, with 
moderate to profuse ear discharge, discharge is mucopurulent, cream in color, not fouling and not 
blood-stained. Each episode is either aggravated by URI(upper respiratory infection) or head bath and 
relieved completely by medications. 
He has some difficulty in hearing in his right ear. He has no history of vertigo, tinnitus, headache, 
fever, vomiting, malaise, lethargy, facial asymmetry, ear pain, retroorbital pain, diplopia, or vision 
problems. 
He has no history of trauma to the ear. He has no nasal obstruction, nasal discharge, sneezing, itching 
over the nose, epistaxis, smell disturbances, throat pain, cough, swallowing difficulty, or breathing 
difficulty